In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import folium
import json
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import warnings
warnings.filterwarnings('ignore')
from shapely.geometry import shape, Point

from matplotlib.patches import Patch
from matplotlib.lines import Line2D

from shapely.ops import unary_union

## **한강 공원 & 서울시 지도**

In [ ]:
geo = gpd.read_file('/content/drive/MyDrive/24-1 데마 플젝/DATA/HangJeongDong_ver20230701.geojson')
seoul = geo[geo['sido']=='11']
seoul

In [ ]:
hangang = gpd.read_file('/content/drive/MyDrive/24-1 데마 플젝/DATA/서울시 주요 115장소 영역.shp')
hangang = hangang[hangang['AREA_NM'].str.contains('한강공원')]
hangang['center_point'] = hangang['geometry'].geometry.centroid
hangang = hangang.reset_index(drop=True)
hangang.head()

In [ ]:
unioned_gdf = seoul.groupby('sggnm')['geometry'].agg(lambda x: x.unary_union).reset_index()
GU = gpd.GeoDataFrame(unioned_gdf, geometry='geometry', crs=seoul.crs)
GU.head()

## **따릉이 대여소**

In [ ]:
bcy = pd.read_csv('/content/drive/MyDrive/24-1 데마 플젝/DATA/서울시 따릉이대여소 마스터 정보.csv', encoding='CP949')
bcy.head()

In [ ]:
bcy = bcy[(bcy['경도'] != 0) & (bcy['위도'] != 0)]
bcy['point'] = bcy.apply(lambda row: Point(row['경도'], row['위도']), axis=1)
bcy = gpd.GeoDataFrame(bcy, geometry='point')
bcy.head()

## **따릉이 최종 시각화**

In [1]:
bcy_dict = {'강서' : ['ST-2493'],
            '광나루' : ['ST-486', 'ST-3098', 'ST-3070', 'ST-2746', 'ST-2609', 'ST-2604', 'ST-1684', 'ST-1627', 'ST-1422', 'ST-1048'],
            '난지' : ['ST-1754'],
            '뚝섬' : ['ST-99', 'ST-980', 'ST-988', 'ST-483', 'ST-2891', 'ST-101', 'ST-2865', 'ST-2316', 'ST-258', 'ST-2546', 'ST-2306',
                      'ST-1663', 'ST-113'],
            '망원' : ['ST-340', 'ST-339', 'ST-2874', 'ST-3', 'ST-2905', 'ST-209'],
            '반포' : ['ST-776', 'ST-749', 'ST-3010', 'ST-1912', 'ST-1930'],
            '양화' : ['ST-426', 'ST-422', 'ST-421', 'ST-3187', 'ST-295', 'ST-278', 'ST-277', 'ST-1980'],
            '여의도' :['ST-73', 'ST-68', 'ST-67', 'ST-65', 'ST-3160', 'ST-64', 'ST-54', 'ST-3239', 'ST-47', 'ST-46', 'ST-414', 'ST-296',
                       'ST-297', 'ST-2814', 'ST-1988', 'ST-2694', 'ST-2647', 'ST-2646', 'ST-2643', 'ST-1987', 'ST-1741', 'ST-1971'],
            '이촌' : ['ST-440', 'ST-3249', 'ST-322', 'ST-3083', 'ST-2866', 'ST-2456', 'ST-2450', 'ST-1444', 'ST-1326', 'ST-2449', 'ST-2448',
                      'ST-2438', 'ST-2240', 'ST-2239', 'ST-2184', 'ST-335', 'ST-1446', 'ST-1343', 'ST-1339'],
            '잠실' : ['ST-542', 'ST-2991', 'ST-2850', 'ST-2728', 'ST-2609', 'ST-2595', 'ST-2594', 'ST-2593', 'ST-1797', 'ST-1705'],
            '잠원' : ['ST-777', 'ST-2673', 'ST-776', 'ST-749', 'ST-2414', 'ST-1930', 'ST-1923']}

In [ ]:
data = {'hangang': [], '대여소_ID': []}

for hangang, ids in bcy_dict.items():
    for id_ in ids:
        data['hangang'].append(hangang)
        data['대여소_ID'].append(id_)

df = pd.DataFrame(data)

In [ ]:
df = df.merge(bcy, how='left', on='대여소_ID')

In [ ]:
df = gpd.GeoDataFrame(df, geometry='point')
df

In [ ]:
df['hangang'].value_counts()

In [ ]:
hangang['count'] = [1, 10, 1, 13, 6, 5, 8, 22, 19, 10, 7]
hangang

In [ ]:
fig, ax = plt.subplots(figsize = (12, 10))
GU.boundary.plot(ax=ax, linewidth=0.6, color='lightgrey')
seoul.plot(linewidth=0.5, color='whitesmoke', ax=ax)
hangang.plot(linewidth=0.2, color='lightblue', ax=ax)
df['point'].plot(marker='o', color='green', markersize=5, alpha=0.3, ax=ax)
ax.axis('off')

In [ ]:
fig, ax = plt.subplots(figsize = (12, 10))
GU.boundary.plot(ax=ax, linewidth=0.6, color='lightgrey')
seoul.plot(linewidth=0.5, color='whitesmoke', ax=ax)
hangang.plot(linewidth=0.2, color='lightblue', ax=ax)
df['point'].plot(marker='o', color='green', markersize=5, alpha=0.3, ax=ax)

for index, row in hangang.iterrows():
    ax.text(row['center_point'].x, row['center_point'].y, str(row['count']),
            fontsize=10, ha='center', va='center', color='black', weight='bold')

ax.axis('off')